In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

import matplotlib.pyplot as plt
import numpy as np

In [3]:
# 演示一维卷积： 一维卷积不代表卷积核只有一维，也不代表被卷积的数据也是一维。一维的意思是说卷积的方向是一维的。
# 参考解释： https://blog.csdn.net/amateurSU/article/details/138557901

batch_size = 2  # 多少批文本数据
word_count = 4  # 每批文本包含多少个单词
embedding_dim = 6 # 每个单词用多少维向量来表示

# 一批数据的张量： 行就是这批文本的各个单词， 列就是每个单词的向量表示
#    I Love My Family
#    *  *   *   *
#    *  *   *   *
#    *  *   *   *
input_data = torch.randn(batch_size, embedding_dim, word_count)

in_channels = embedding_dim  # 每个单词的向量维度

# 比如对于如下一批数据， 如果out_channels为5，则会用5个不同的卷积核与这批数据做卷积运算
#  [[ 0.2605, -1.5650, -0.5183, -0.4161],
#   [ 0.3433,  1.5320,  2.0031,  0.3885],
#   [ 2.3063, -0.3679,  0.7549, -0.9822],
#   [ 1.5229,  1.4932, -0.4872,  0.7238],
#   [ 0.6817,  0.1782, -0.5580, -1.0454],
#   [ 1.0688, -1.6322,  0.9375, -0.3195]]

# 卷积核的高度是 in_channels，宽度是kernel_size

# 根据填充padding为1， 步长stride为2，则：
# 第1个卷积核运算结果为：  [0.0839, 0.0000],
# 第2个卷积核运算结果为：  [0.3262, 0.2788],
# 第3个卷积核运算结果为：  [0.0000, 0.0683],
# 第4个卷积核运算结果为：  [0.0000, 0.0000],
# 第5个卷积核运算结果为：  [0.6557, 1.5359],

# 最后把5个卷积核结果 拼接起来就是 5*2维度的矩阵
out_channels = 5  # 卷积的输出通道，也就是有多少个卷积核

kernel_size = 3  # 卷积核的大小，此处指宽度， 实际高度是 in_channels, 也就是卷积核其实是二维的

stride = 2  # 步幅
padding = 1 # 左右填充大小

# 沿着句子中各个单词的方向进行卷积运算， 即input_data的第三个维度
conv1d_layer = nn.Conv1d(in_channels, out_channels, kernel_size, stride, padding)
relu = nn.ReLU()

conv1d_output = conv1d_layer(input_data)
output = relu(conv1d_output)  # 应用激活函数

# in_channels -> out_channel的变换
print('input size: ', input_data.shape)
print(input_data)
print('output size: ', output.shape) #stride 和 padding参数影响输出的第三个维度
print(output)

input size:  torch.Size([2, 6, 4])
tensor([[[ 0.8106, -1.8630, -0.3909, -0.8853],
         [-1.4399, -0.7427, -2.3332,  1.2521],
         [-0.0532, -2.2048, -0.2329,  1.8016],
         [ 0.2515,  0.9480, -0.7505,  0.9304],
         [-0.8553,  0.1588, -0.2515,  0.5496],
         [-0.6063, -1.1989, -1.6598, -1.3802]],

        [[ 1.2116,  0.1249,  0.3586,  2.4219],
         [ 0.4004,  0.6287,  0.1993,  0.3761],
         [-1.6159, -0.1262, -0.1153,  0.4964],
         [ 0.5437, -1.8344, -0.5083,  0.9897],
         [-1.1565, -0.6082,  1.1919,  2.2440],
         [ 0.0358,  0.1542, -0.4149, -0.0404]]])
output size:  torch.Size([2, 5, 2])
tensor([[[0.4945, 0.0000],
         [0.0000, 0.0000],
         [0.0000, 0.1120],
         [1.3417, 1.3135],
         [0.0000, 0.0000]],

        [[0.0000, 0.0000],
         [0.1645, 0.3309],
         [0.4919, 0.0000],
         [0.0000, 0.4103],
         [0.0000, 0.4062]]], grad_fn=<ReluBackward0>)


In [11]:
# 演示二维卷积: in_channels, out_channels, kernel_size
in_channels = 2
input_data = torch.arange(49 * in_channels,dtype=torch.float32).view(1, in_channels, 7, 7) # 四维数据， 第一维表示批次， 第二维输入通道
conv2d_layer = nn.Conv2d(in_channels = in_channels, out_channels = 1, kernel_size = 3)
#conv2d_layer = nn.Conv2d(in_channels = in_channels, out_channels = 1, kernel_size = (3, 3))

# out_channels = 2表示有2个卷积核，每个卷积核大小 in_channels * kernel_size * kernel_size
conv2d_layer2 = nn.Conv2d(in_channels = in_channels, out_channels = 2, kernel_size = 3)

output = conv2d_layer(input_data)
print(input_data.shape)
print(input_data)
print('\n', output.shape)
print(output)
print(list(conv2d_layer.parameters())) # 卷积核权重

output2 = conv2d_layer2(input_data)
print('\n', output2.shape)
print(output2)
print(list(conv2d_layer2.parameters())) # 卷积核权重

torch.Size([1, 2, 7, 7])
tensor([[[[ 0.,  1.,  2.,  3.,  4.,  5.,  6.],
          [ 7.,  8.,  9., 10., 11., 12., 13.],
          [14., 15., 16., 17., 18., 19., 20.],
          [21., 22., 23., 24., 25., 26., 27.],
          [28., 29., 30., 31., 32., 33., 34.],
          [35., 36., 37., 38., 39., 40., 41.],
          [42., 43., 44., 45., 46., 47., 48.]],

         [[49., 50., 51., 52., 53., 54., 55.],
          [56., 57., 58., 59., 60., 61., 62.],
          [63., 64., 65., 66., 67., 68., 69.],
          [70., 71., 72., 73., 74., 75., 76.],
          [77., 78., 79., 80., 81., 82., 83.],
          [84., 85., 86., 87., 88., 89., 90.],
          [91., 92., 93., 94., 95., 96., 97.]]]])

 torch.Size([1, 1, 5, 5])
tensor([[[[22.8329, 23.4452, 24.0576, 24.6699, 25.2823],
          [27.1194, 27.7317, 28.3441, 28.9565, 29.5688],
          [31.4059, 32.0183, 32.6306, 33.2430, 33.8554],
          [35.6924, 36.3048, 36.9172, 37.5295, 38.1419],
          [39.9790, 40.5913, 41.2037, 41.8161, 42.4284]]]

In [5]:
# 演示二维卷积: padding, padding_mode
img = torch.arange(25, dtype=torch.float32).reshape(1,1,5,5)
print(img)

# 周围填充0
conv_zeros = nn.Conv2d(in_channels=1, out_channels=1, kernel_size=1, bias=False, padding=3, padding_mode='zeros')
conv_zeros.weight = nn.parameter.Parameter(torch.ones((1,1,1,1)))
img_zeros = conv_zeros(img)
print(img_zeros)

# 以边界对称的点来填充
conv_reflect = nn.Conv2d(in_channels=1, out_channels=1, kernel_size=1, bias=False, padding=3, padding_mode='reflect')
conv_reflect.weight = nn.parameter.Parameter(torch.ones((1,1,1,1)))
img_reflect = conv_reflect(img)
print(img_reflect)

# 复制边界的点填充
conv_replicate = nn.Conv2d(in_channels=1, out_channels=1, kernel_size=1, bias=False, padding=3, padding_mode='replicate')
conv_replicate.weight = nn.parameter.Parameter(torch.ones((1,1,1,1)))
img_replicate = conv_replicate(img)
print(img_replicate)

# 循环复制边界另一侧的点来填充
conv_circular = nn.Conv2d(in_channels=1, out_channels=1, kernel_size=1, bias=False, padding=3, padding_mode='circular')
conv_circular.weight = nn.parameter.Parameter(torch.ones((1,1,1,1)))
img_circular = conv_circular(img)
print(img_circular)

tensor([[[[ 0.,  1.,  2.,  3.,  4.],
          [ 5.,  6.,  7.,  8.,  9.],
          [10., 11., 12., 13., 14.],
          [15., 16., 17., 18., 19.],
          [20., 21., 22., 23., 24.]]]])
tensor([[[[ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
          [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
          [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
          [ 0.,  0.,  0.,  0.,  1.,  2.,  3.,  4.,  0.,  0.,  0.],
          [ 0.,  0.,  0.,  5.,  6.,  7.,  8.,  9.,  0.,  0.,  0.],
          [ 0.,  0.,  0., 10., 11., 12., 13., 14.,  0.,  0.,  0.],
          [ 0.,  0.,  0., 15., 16., 17., 18., 19.,  0.,  0.,  0.],
          [ 0.,  0.,  0., 20., 21., 22., 23., 24.,  0.,  0.,  0.],
          [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
          [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
          [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.]]]],
       grad_fn=<ConvolutionBackward0>)
tensor([[[[18., 17., 16., 15., 16

In [6]:
# 演示二维卷积: groups
# 对输入通道进行分组，节省一半参数量
in_channels = 4
model = nn.Conv2d(in_channels, out_channels = 8, kernel_size = 1, stride = 1, groups = 2, bias = False)

inputs = torch.ones(1, in_channels, 1, 1)
print(inputs)

# 设置卷积核的权重，也就是对应 out_channels个filter，每个filter对应 in_channels / groups 个卷积核
# 1  2
# 3  4
# 5  6
# 7  8
# 9  10
# 11 12
# 12 13
# 13 14
# 15 16
for param in model.parameters():
    print(param.size())
    param.data = torch.FloatTensor([list(range(1, 17))]).view(8,2,1,1)

output = model(inputs)
print(output)


tensor([[[[1.]],

         [[1.]],

         [[1.]],

         [[1.]]]])
torch.Size([8, 2, 1, 1])
tensor([[[[ 3.]],

         [[ 7.]],

         [[11.]],

         [[15.]],

         [[19.]],

         [[23.]],

         [[27.]],

         [[31.]]]], grad_fn=<ConvolutionBackward0>)
